In [1]:
import numpy as np
import tensorflow as tf
import sklearn as sk
import sklearn.linear_model as sklm
import sklearn.metrics as skmt
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io
import h5py
import sys
import gc

sys.path.append('Metric/')
sys.path.append('../Visualization/')
sys.path.append('../Data_Preprocessing//')
from Metric import *
from Visualization import *
from Data_Extractor import *
from sklearn.externals import joblib

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# settings
path_image_dir = "../Data/090085/"
path_road_mask = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif"

classifier_type = 'FCN'
path_model_dir = "./Segmentation-FCN/Result/Inception/Incep_3-32;1-32|3-64;1-64_m_weight_bn_p0_e20_r0/"
model_name = "Incep_3-32;1-32|3-64;1-64_m_weight_bn_p0_e20_r0"
size = 128
step = 16

path_pred_dir = "./Time-Series/Pred/"


block_size = 128

use_norm = True
if model_name.find("_G") > 0:
    norm = "Gaussian"
elif model_name.find("_m") > 0:
    norm = "mean"
else:
    norm = None
    use_norm = False

image_list = ['090085_20170531.h5',
              
#               '090085_20160512.h5',
              '090085_20160528.h5',
#               '090085_20160512.h5',# cloudy
              
#               '090085_20150510.h5', # little bit cloudy
#               '090085_20150526.h5', => too much cloud
              '090085_20150611.h5',

#               '090085_20140320.h5',
#               '090085_20140405.h5',
#               '090085_20140421.h5',
#               '090085_20140507.h5',# cloudy
#               '090085_20140523.h5', => snow cover + too much cloud
#               '090085_20140608.h5',
#               '090085_20140624.h5',
#                   '090085_20140710.h5', # might be alright...
              '090085_20140726.h5',
              
              '090085_20130520.h5']
image_list = sorted(image_list)

In [ ]:
# load raw image
for image_name in image_list:
    print(image_name)
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()

    show_raw_image(raw_image, size=-1)
plt.close()
gc.collect()

In [4]:
class Classifier ():
    def __init__(self, path, name, classifier_type):
        assert classifier_type in set(['LR', 'FCN'])
        self.classifier_type = classifier_type
        if classifier_type == 'LR':
            self.classifier = joblib.load(path+name)
            self.pos_idx = int(np.where(self.classifier.classes_ == 1)[0])
            self.predict = lambda patch: self.classifier.predict_proba(patch.reshape((1, -1)))[0,self.pos_idx]

        else: # FCN
            tf.reset_default_graph()
            tf.train.import_meta_graph(path+name+'.meta')
            sess = tf.InteractiveSession()
            saver = tf.train.Saver()
            saver.restore(sess, path+name)
            
            graph = tf.get_default_graph()
            self.x = graph.get_tensor_by_name("input/x:0")
            self.is_training = graph.get_tensor_by_name("input/is_training:0")
            self.prob_out = graph.get_tensor_by_name("prob_out/prob_out:0")
            
            self.predict = lambda patch: self.prob_out.eval(feed_dict={self.x: patch.transpose((0, 2, 3, 1)), 
                                                                       self.is_training: False})[0,:,:]

In [5]:
# re-load classifier
if classifier_type == 'LR':
    is_valid = lambda patch: (patch != -9999).all()
    
else:
    assert classifier_type == 'FCN'
    is_valid = lambda patch: ((patch==-9999).sum() / np.prod(np.array(patch.shape))) < 1/100

classifier = Classifier(path_model_dir, model_name, classifier_type)

INFO:tensorflow:Restoring parameters from ./Segmentation-FCN/Result/Inception/Incep_3-32;1-32|3-64;1-64_m_weight_bn_p0_e20_r0/Incep_3-32;1-32|3-64;1-64_m_weight_bn_p0_e20_r0


In [7]:
path_pred_in_time = path_pred_dir + model_name + '.h5'
print(path_pred_in_time)

for image_name in image_list:
    gc.collect()

    date = image_name.split('.')[0].split('_')[-1]
    image_path = path_image_dir + image_name

    # check prediction not existing
    h5f = h5py.File(path_pred_in_time, 'a')
    if date in set([pred for pred in h5f.keys()]):
        print(image_path, 'already predicted, skip')
        h5f.close()
        continue
    h5f.close()

    # load raw image
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()
    
    # construct extractor
    data_extractor = Pred_Data_Extractor(raw_image, step=step, size=size,
                                         normalization=norm, is_valid=is_valid)
    
    # pred on image
    if classifier_type == 'LR':
        pred_road = np.zeros(shape=raw_image[0].shape)
        print(pred_road.shape, image_name, date)

        for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=use_norm):
            pred_road[coord[0]+int(size/2),coord[1]+int(size/2)] = classifier.predict(patch)

    else: # FCN        
        pred_road = np.zeros(shape=(raw_image.shape[1], raw_image.shape[2], 2))
        print(pred_road.shape, image_name, date)

        for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=use_norm):
            pred_road[coord[0]:coord[0]+size, coord[1]:coord[1]+size] += classifier.predict(patch)
            
    # save prediction
    h5f = h5py.File(path_pred_in_time, 'a')
    print([i for i in h5f.items()])
    h5f.create_dataset(name=date, data=pred_road)
    h5f.close()

./Time-Series/Pred/Incep_3-32;1-32|3-64;1-64_m_weight_bn_p0_e20_r0.h5
mu =  [-0.00193307 -0.00129789  0.00198825  0.00165098 -0.00051675  0.00051223
  0.00119147]
(7961, 8091, 2) 090085_20130520.h5 20130520
[]
mu =  [-0.00177715 -0.00112118 -0.00110293 -0.00525456  0.00438297 -0.0016226
  0.00298007]
(7961, 8091, 2) 090085_20140726.h5 20140726
[('20130520', <HDF5 dataset "20130520": shape (7961, 8091, 2), type "<f8">)]
mu =  [ 0.00033616  0.00113001  0.00035501 -0.00320866 -0.00216324 -0.00197344
 -0.0015663 ]
(7961, 8091, 2) 090085_20150611.h5 20150611
[('20130520', <HDF5 dataset "20130520": shape (7961, 8091, 2), type "<f8">), ('20140726', <HDF5 dataset "20140726": shape (7961, 8091, 2), type "<f8">)]
mu =  [-0.00166324  0.0017417  -0.00267226 -0.00253265 -0.00369252  0.00222735
 -0.00384184]
(7961, 8091, 2) 090085_20160528.h5 20160528
[('20130520', <HDF5 dataset "20130520": shape (7961, 8091, 2), type "<f8">), ('20140726', <HDF5 dataset "20140726": shape (7961, 8091, 2), type "<f8">

Analyze pred along the time

In [ ]:
def get_patch(coord, mask, size):
    return mask[coord[0]:coord[0]+size, coord[1]:coord[1]+size]

def pred_normalization(pred):
    pred_norm = pred[:,:,1]/pred.sum(axis=-1)
    pred_norm[np.where(pred_norm != pred_norm)] = 0
    pred_norm[np.where(pred_norm == np.float('inf'))] = 1
    return pred_norm

# softmax
def pred_softmax(pred):
    threshold = 500
    pred_exp = pred.copy()
    inf_idx = np.where(pred_exp > threshold)
    
    for x, y in zip(inf_idx[0], inf_idx[1]):
        while((pred_exp[x,y] > threshold).any()):
            pred_exp[x,y] = pred_exp[x,y] / 10
    pred_exp = np.exp(pred_exp[:,:,1])/np.exp(pred_exp).sum(axis=-1)
    pred_exp[np.where(pred[:,:,1] == 0)] = 0 # softmax([0,0]) = (0.5, 0.5)
    return pred_exp

In [ ]:
# load road mask
road_mask = skimage.io.imread(path_road_mask)
road_proba = {}

for image_name in image_list:
    gc.collect()

    # restore prediction 
    date = image_name.split('.')[0].split('_')[-1]
    h5f = h5py.File(path_pred_in_time, 'r')
    pred_road = np.array(h5f[date])
    h5f.close()

    if classifier_type == 'FCN':
        print('normalizing pred')
        pred_road = pred_normalization(pred_road)
    
    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()
    
    # construct extractor => scane through the image
    data_extractor = Pred_Data_Extractor(raw_image, step=block_size, size=block_size,
                                         normalization=None, is_valid=lambda patch: True)
    assert not (date in road_proba)
    road_proba[date] = {}
    for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=False):
        sub_valid_mask = (patch[0][0] != -9999)
        sub_road_mask  = (get_patch(coord, road_mask, block_size) == 1)
        sub_valid_road_mask = np.logical_and(sub_valid_mask, sub_road_mask)
        
        # patch should contain some roads
        if sub_road_mask.sum() / np.prod(sub_road_mask.shape) < 0.1: continue
#         if sub_valid_road_mask.sum() / np.prod(sub_valid_road_mask.shape) > 0.25:        

        assert not (coord in road_proba[date])
        mean_prob = get_patch(coord, pred_road, block_size)[np.where(sub_valid_road_mask)].mean()
        road_proba[date][coord] = mean_prob

In [ ]:
# get coord_list
coord_list = sorted(road_proba[date].keys())
for image_name in image_list:
    date = image_name.split('.')[0].split('_')[-1]
    assert ( coord_list == sorted(road_proba[date].keys()) )

# get date_list
date_list = sorted(road_proba.keys())

# construct matrix [date][coord]
sorted_road = np.array([[road_proba[date][coord] for coord in coord_list]
                        for date in date_list])

In [ ]:
# plot proba by date
plt.figure(figsize=(20, 20))
for row, coord in zip((sorted_road.T), coord_list):
    valid_row = row[np.where(row == row)]
    
    # ensure the probability increase
    if len(valid_row) > 0 and (np.diff(valid_row) >= 0).all() and (np.diff(valid_row) > 0).any():
        plt.plot(row, label=str(coord))
        
plt.legend()
plt.show()
plt.close()

In [ ]:
appearing_road_coord_list = []
for row, coord in zip((sorted_road.T), coord_list):
    valid_row = row[np.where(row == row)]
    
    # ensure the probability increase
    if len(valid_row) > 0 and (np.diff(valid_row) >= 0).all() and (np.diff(valid_row) > 0).any():
        appearing_road_coord_list.append(coord)
        
print(len(appearing_road_coord_list), set(appearing_road_coord_list))

In [ ]:
nrows = len(appearing_road_coord_list)
ncols = len(image_list)
print(nrows, ncols)

plt.close('all')
plt.figure(figsize=(60,240))

# from 2014 to 2017
for image_name, index in zip(sorted(image_list), range(1,nrows+1)):
    
    # restore prediction 
    date = image_name.split('.')[0].split('_')[-1]
    h5f = h5py.File(path_pred_in_time, 'r')
    pred_road = np.array(h5f[date])
    h5f.close()

    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()
    
    gc.collect()
    
    # plot the scene in current date
    for coord in appearing_road_coord_list:
        plt.title(str(date) + ',' + str(coord))
        plt.subplot(nrows, ncols, index)
        show_pred_prob_with_raw(raw_image, pred_road, true_road=road_mask, 
                                coord=coord, size=block_size, show_plot=False)
        index += ncols

plt.savefig('changing_road')
plt.show()
        

In [ ]:
plt.close('all')
gc.collect()

In [ ]:
road_proba_divide = {}
divide_num = 4
step_size = int(block_size/divide_num)

for image_name in image_list:
    gc.collect()

    # restore prediction 
    date = image_name.split('.')[0].split('_')[-1]
    h5f = h5py.File(path_pred_in_time, 'r')
    pred_road = np.array(h5f[date])
    h5f.close()

    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()

    assert not (date in road_proba_divide)
    
    for coord in coord_list:
        for offset_x in range(divide_num):
            offset_x *= step_size

            for offset_y in range(divide_num):
                offset_y *= step_size
                
                coord[0] += offset_x
                coord[1] += offset_y
                
                sub_valid_mask = (get_patch(coord, raw_image[0], step_size) != -9999)
                sub_road_mask  = (get_patch(coord, road_mask, block_size) == 1)
                sub_valid_road_mask = np.logical_and(sub_valid_mask, sub_road_mask)
                
                if sub_valid_road_mask.sum() / np.prod(sub_valid_road_mask.shape) <  1/100:
                    

In [ ]:
sub_valid_road_mask.sum() / np.prod(sub_valid_road_mask.shape) < 1/100


In [ ]:
# load road mask
road_mask = skimage.io.imread(path_road_mask)
road_proba = {}

for image_name in image_list:
    gc.collect()

    # restore prediction 
    date = image_name.split('.')[0].split('_')[-1]
    h5f = h5py.File(path_pred_in_time, 'r')
    pred_road = np.array(h5f[date])
    h5f.close()

    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()
    
    # construct extractor
    data_extractor = Pred_Data_Extractor(raw_image, step=block_size, size=block_size,
                                         normalization=None, is_valid=lambda patch: True)
    assert not (date in road_proba)
    road_proba[date] = {}
    for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=False):
        sub_valid_mask = patch[0][0] != -9999
        sub_road_mask  = get_patch(coord, road_mask, block_size) == 1
        sub_valid_road_mask = np.logical_and(sub_valid_mask, sub_road_mask)
        
        assert not (coord in road_proba[date])
        mean_prob = get_patch(coord, pred_road, block_size)[np.where(sub_valid_road_mask)].mean()
        if mean_prob != mean_prob:
            mean_prob = 0
        road_proba[date][coord] = mean_prob

Analyze log pred

In [ ]:
log_pred = -np.log(-pred_road + 1 + 1e-7)
print(log_pred.min(), log_pred.max(), log_pred.mean())

norm_log_pred = (log_pred - log_pred.min()) / (log_pred.max()-log_pred.min())
print(norm_log_pred.min(), norm_log_pred.max(), norm_log_pred.mean())

plt.hist(x=norm_log_pred.flatten(), bins=100, histtype='step')
plt.show()
plt.close()
plt.hist(x=norm_log_pred[np.where(norm_log_pred>0)].flatten(), bins=100, histtype='step')
plt.show()
plt.close()

In [ ]:
path_road = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif"
road_mask = skimage.io.imread(path_road)

In [ ]:
show_pred_road_against_raw(raw_image, norm_log_pred, road_mask, threshold=0.40, figsize=(150,150), show_plot=False,
                        save_path="123")
gc.collect()

In [ ]:
plt.imshow(np.array([[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]]), cmap='hot')
plt.show()
plt.figure(figsize=(100,100))
plt.imshow(pred_road, cmap=plt.get_cmap('hot'))
plt.show()
plt.close()

In [ ]:
plt.imshow(np.array([[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]]), cmap='hot')
plt.show()
plt.figure(figsize=(100,100))
plt.imshow(np.log(pred_road + 1e-9), cmap='hot')
plt.show()
plt.close()